In [1]:
import os
import pandas as pd
from unidecode import unidecode
from dotenv import load_dotenv
import requests, zipfile
from io import StringIO, BytesIO

In [2]:
load_dotenv()

True

# Extração e transformação inicial

Durante a evolução do projeto, também foi sugerida a inclusão do IDEB do município como indicador educacional ao dashboard de visão geral. Os dados são divulgados de maneira agregada para o município separados por anos iniciais, anos finais e ensino médio. Vamos obtê-los diretamente do site do INEP.

In [3]:
def get_ideb(ciclo: str) -> pd.DataFrame:
    """
    Baixa e processa dados do IDEB para o município de São Paulo.
    
    Parâmetros:
    -----------
    ciclo : str
        O ciclo educacional desejado. Deve ser um dos valores:
        - 'anos_iniciais': Anos iniciais do ensino fundamental
        - 'anos_finais': Anos finais do ensino fundamental  
        - 'ensino_medio': Ensino médio
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame com os dados do IDEB filtrados para o município de São Paulo
        (código 3550308).
    
    Levanta:
    --------
    ValueError
        Se o parâmetro 'ciclo' não for um dos valores válidos.
    
    Exemplo:
    --------
    >>> df_ideb = get_ideb('anos_iniciais')
    >>> print(df_ideb.head())
    """
    if ciclo not in ['anos_iniciais', 'anos_finais', 'ensino_medio']:
        raise ValueError("Ciclo deve ser 'anos_iniciais', 'anos_finais' ou 'ensino_medio'")
    
    url = f'https://download.inep.gov.br/ideb/resultados/divulgacao_{ciclo}_municipios_2023.zip'
    with requests.get(url) as r:
        with zipfile.ZipFile(BytesIO(r.content)) as z:
            file_path = f'divulgacao_{ciclo}_municipios_2023/divulgacao_{ciclo}_municipios_2023.xlsx'
            df = pd.read_excel(z.open(file_path), skiprows=9)

    df = df[df['CO_MUNICIPIO'] == 3550308]
    return df

In [4]:
df_ideb_iniciais = get_ideb('anos_iniciais')
df_ideb_iniciais

,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,VL_APROVACAO_2005_SI_4,VL_APROVACAO_2005_SI,VL_APROVACAO_2005_1,VL_APROVACAO_2005_2,VL_APROVACAO_2005_3,VL_APROVACAO_2005_4,...,VL_OBSERVADO_2021,VL_OBSERVADO_2023,VL_PROJECAO_2007,VL_PROJECAO_2009,VL_PROJECAO_2011,VL_PROJECAO_2013,VL_PROJECAO_2015,VL_PROJECAO_2017,VL_PROJECAO_2019,VL_PROJECAO_2021
9940,SP,3550308.0,São Paulo,Federal,-,-,-,-,-,-,...,-,6.5,-,-,5.7,6,6.2,6.5,6.7,6.9
9941,SP,3550308.0,São Paulo,Estadual,95.1,-,95.2,97.1,97.2,91,...,6,6.1,4.7,5,5.4,5.7,5.9,6.2,6.4,6.6
9942,SP,3550308.0,São Paulo,Municipal,94.7,-,96.5,98.1,98.3,87.6,...,5.7,5.6,4.1,4.5,4.9,5.2,5.4,5.7,6,6.2
9943,SP,3550308.0,São Paulo,Pública,94.9,-,95.8,97.5,97.7,89.3,...,5.9,5.9,4.4,4.7,5.1,5.4,5.7,5.9,6.2,6.4


In [5]:
df_ideb_finais = get_ideb('anos_finais')
df_ideb_finais

,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,VL_APROVACAO_2005_SI_4,VL_APROVACAO_2005_1,VL_APROVACAO_2005_2,VL_APROVACAO_2005_3,VL_APROVACAO_2005_4,VL_INDICADOR_REND_2005,...,VL_OBSERVADO_2021,VL_OBSERVADO_2023,VL_PROJECAO_2007,VL_PROJECAO_2009,VL_PROJECAO_2011,VL_PROJECAO_2013,VL_PROJECAO_2015,VL_PROJECAO_2017,VL_PROJECAO_2019,VL_PROJECAO_2021
10053,SP,3550308.0,São Paulo,Estadual,85.5,88.9,88.6,87.4,76.9,0.851411,...,5.1,4.9,3.8,4,4.3,4.7,5,5.3,5.5,5.8
10054,SP,3550308.0,São Paulo,Municipal,93.6,96.5,95.7,95,86,0.930964,...,5.1,4.8,4.1,4.3,4.6,5,5.3,5.6,5.8,6
10055,SP,3550308.0,São Paulo,Pública,88.8,92.2,91.6,90.5,80.3,0.883665,...,5.1,4.8,4,4.1,4.4,4.8,5.2,5.4,5.6,5.9


In [6]:
df_ideb_medio = get_ideb('ensino_medio')
df_ideb_medio

,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,VL_APROVACAO_2017_SI_4,VL_APROVACAO_2017_1,VL_APROVACAO_2017_2,VL_APROVACAO_2017_3,VL_APROVACAO_2017_4,VL_INDICADOR_REND_2017,...,VL_NOTA_MEDIA_2021,VL_NOTA_MATEMATICA_2023,VL_NOTA_PORTUGUES_2023,VL_NOTA_MEDIA_2023,VL_OBSERVADO_2017,VL_OBSERVADO_2019,VL_OBSERVADO_2021,VL_OBSERVADO_2023,VL_PROJECAO_2019,VL_PROJECAO_2021
8058,SP,3550308.0,São Paulo,Estadual,81.6,74.2,82.2,90.5,-,0.817611,...,4.713662,265.35,277.16,4.565442,3.5,3.9,4.3,4.1,3.7,3.9
8059,SP,3550308.0,São Paulo,Federal,88.4,85.2,86.8,96.8,-,0.893165,...,6.597057,327.41,330.84,6.240664,6.3,6.7,6,6,6.5,6.7
8060,SP,3550308.0,São Paulo,Municipal,85.3,79.3,84.8,92.1,93.3,0.869969,...,4.711535,266.21,273.1,4.516742,3.5,3.7,4.5,4.3,3.7,4
8061,SP,3550308.0,São Paulo,Pública,81.7,74.3,82.3,90.5,93.3,0.844283,...,4.727806,265.86,277.57,4.578743,3.6,4.1,4.4,4.2,3.8,4


# Transformação e mesclagem de dados

Primeiro, vamos concatenar os dados dos três níveis de ensino em um único DataFrame, adicionando uma coluna para identificar o nível de ensino correspondente.

In [7]:
df_ideb = pd.concat(
    [df_ideb_iniciais
    .assign(CICLO='Anos Iniciais'),
    df_ideb_finais
    .assign(CICLO='Anos Finais'),
    df_ideb_medio
    .assign(CICLO='Ensino Médio')]
)

df_ideb

,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,VL_APROVACAO_2005_SI_4,VL_APROVACAO_2005_SI,VL_APROVACAO_2005_1,VL_APROVACAO_2005_2,VL_APROVACAO_2005_3,VL_APROVACAO_2005_4,...,VL_OBSERVADO_2023,VL_PROJECAO_2007,VL_PROJECAO_2009,VL_PROJECAO_2011,VL_PROJECAO_2013,VL_PROJECAO_2015,VL_PROJECAO_2017,VL_PROJECAO_2019,VL_PROJECAO_2021,CICLO
9940,SP,3550308.0,São Paulo,Federal,-,-,-,-,-,-,...,6.5,-,-,5.7,6,6.2,6.5,6.7,6.9,Anos Iniciais
9941,SP,3550308.0,São Paulo,Estadual,95.1,-,95.2,97.1,97.2,91,...,6.1,4.7,5,5.4,5.7,5.9,6.2,6.4,6.6,Anos Iniciais
9942,SP,3550308.0,São Paulo,Municipal,94.7,-,96.5,98.1,98.3,87.6,...,5.6,4.1,4.5,4.9,5.2,5.4,5.7,6,6.2,Anos Iniciais
9943,SP,3550308.0,São Paulo,Pública,94.9,-,95.8,97.5,97.7,89.3,...,5.9,4.4,4.7,5.1,5.4,5.7,5.9,6.2,6.4,Anos Iniciais
10053,SP,3550308.0,São Paulo,Estadual,85.5,NaN,88.9,88.6,87.4,76.9,...,4.9,3.8,4,4.3,4.7,5,5.3,5.5,5.8,Anos Finais
10054,SP,3550308.0,São Paulo,Municipal,93.6,NaN,96.5,95.7,95,86,...,4.8,4.1,4.3,4.6,5,5.3,5.6,5.8,6,Anos Finais
10055,SP,3550308.0,São Paulo,Pública,88.8,NaN,92.2,91.6,90.5,80.3,...,4.8,4,4.1,4.4,4.8,5.2,5.4,5.6,5.9,Anos Finais
8058,SP,3550308.0,São Paulo,Estadual,NaN,NaN,NaN,NaN,NaN,NaN,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,3.7,3.9,Ensino Médio
8059,SP,3550308.0,São Paulo,Federal,NaN,NaN,NaN,NaN,NaN,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.7,Ensino Médio
8060,SP,3550308.0,São Paulo,Municipal,NaN,NaN,NaN,NaN,NaN,NaN,...,4.3,NaN,NaN,NaN,NaN,NaN,NaN,3.7,4,Ensino Médio


Agora, filtramos apenas a rede municipal.

In [8]:
df_ideb = df_ideb[df_ideb['REDE'] == 'Municipal'].reset_index(drop=True)
df_ideb

,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,REDE,VL_APROVACAO_2005_SI_4,VL_APROVACAO_2005_SI,VL_APROVACAO_2005_1,VL_APROVACAO_2005_2,VL_APROVACAO_2005_3,VL_APROVACAO_2005_4,...,VL_OBSERVADO_2023,VL_PROJECAO_2007,VL_PROJECAO_2009,VL_PROJECAO_2011,VL_PROJECAO_2013,VL_PROJECAO_2015,VL_PROJECAO_2017,VL_PROJECAO_2019,VL_PROJECAO_2021,CICLO
0,SP,3550308.0,São Paulo,Municipal,94.7,-,96.5,98.1,98.3,87.6,...,5.6,4.1,4.5,4.9,5.2,5.4,5.7,6,6.2,Anos Iniciais
1,SP,3550308.0,São Paulo,Municipal,93.6,NaN,96.5,95.7,95,86,...,4.8,4.1,4.3,4.6,5,5.3,5.6,5.8,6,Anos Finais
2,SP,3550308.0,São Paulo,Municipal,NaN,NaN,NaN,NaN,NaN,NaN,...,4.3,NaN,NaN,NaN,NaN,NaN,NaN,3.7,4,Ensino Médio


Por último, mantemos apenas as colunas relevantes para o nosso dashboard e renomeamos as colunas para um formato mais amigável.

In [9]:
df_ideb = df_ideb[['CICLO', 'VL_OBSERVADO_2023']]
df_ideb


,CICLO,VL_OBSERVADO_2023
0,Anos Iniciais,5.6
1,Anos Finais,4.8
2,Ensino Médio,4.3


In [10]:
df_ideb = df_ideb.assign(IDEB_2023=pd.to_numeric(df_ideb['VL_OBSERVADO_2023']))
df_ideb = df_ideb.drop(columns='VL_OBSERVADO_2023')
df_ideb

,CICLO,IDEB_2023
0,Anos Iniciais,5.6
1,Anos Finais,4.8
2,Ensino Médio,4.3


# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [11]:
base_path = os.path.join('data_output', 'educacao')

if not os.path.exists(base_path):
    os.makedirs(base_path)

filepath = os.path.join(base_path, f'ideb-municipal.csv')

df_ideb.to_csv(filepath,
               index=False,
               sep=';',
               decimal=',',
               encoding='utf-8')